In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [2]:
class Model(tfrs.Model):

  def __init__(self, model):
    super().__init__()
    self._model = model
    self._logit_layer = tf.keras.layers.Dense(1)

    self.task = tfrs.tasks.Ranking(
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[
        tf.keras.metrics.RootMeanSquaredError("RMSE")
      ]
    )

  def call(self, x):
    x = self._model(x)
    return self._logit_layer(x)

  def compute_loss(self, features, training=False):
    x, labels = features
    scores = self(x)

    return self.task(
        labels=labels,
        predictions=scores,
    )

In [3]:
crossnet = Model(tfrs.layers.dcn.Cross())
deepnet = Model(
    tf.keras.Sequential([
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(128, activation="relu")
    ])
)

In [4]:
import pymysql
import pandas as pd

host = '127.0.0.1'
port = 3306
user = 'root'
password = 'acorn1234'
database = 'music'

connection = pymysql.connect(host=host, port=port, user=user, password=password, database=database)

new_song = pd.read_sql_query('SELECT * FROM new_song', connection)

connection.close()


C:\Users\401-14\AppData\Local\Temp\ipykernel_30936\1382598649.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_song = pd.read_sql_query('SELECT * FROM new_song', connection)


In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
# new_song 데이터프레임에서 'age', 'gender', 'like' 열을 각각 가져옵니다.
age_column = new_song['age'].values
gender_column = new_song['gender'].values
score_column = new_song['sentiment_score'].values

# age, gender, like 열을 합쳐서 train_x에 저장합니다.
train_x = tf.stack([age_column, gender_column], axis=1)

# like 열을 train_y에 저장합니다.
train_y = score_column

# train_x1,train_y1,test_x,test_y=train_test_split(train_x,train_y,test_size=0.2,random_state=42)
# 학습 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(1000)

In [6]:
epochs = 10
learning_rate = 0.1

In [7]:
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32), y))

crossnet.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate))
crossnet.fit(train_dataset, epochs=epochs, verbose=True)

Epoch 1/10
10/10 [==============================] - 0s 1ms/step - RMSE: 1.1845 - loss: 1.4032 - regularization_loss: 0.0000e+00 - total_loss: 1.4032
Epoch 2/10
10/10 [==============================] - 0s 1ms/step - RMSE: 1.1412 - loss: 1.3123 - regularization_loss: 0.0000e+00 - total_loss: 1.3123
Epoch 3/10
10/10 [==============================] - 0s 1ms/step - RMSE: 1.1361 - loss: 1.3001 - regularization_loss: 0.0000e+00 - total_loss: 1.3001
Epoch 4/10
10/10 [==============================] - 0s 1ms/step - RMSE: 1.1341 - loss: 1.2953 - regularization_loss: 0.0000e+00 - total_loss: 1.2953
Epoch 5/10
10/10 [==============================] - 0s 889us/step - RMSE: 1.1329 - loss: 1.2924 - regularization_loss: 0.0000e+00 - total_loss: 1.2924
Epoch 6/10
10/10 [==============================] - 0s 889us/step - RMSE: 1.1319 - loss: 1.2901 - regularization_loss: 0.0000e+00 - total_loss: 1.2901
Epoch 7/10
10/10 [==============================] - 0s 889us/step - RMSE: 1.1311 - loss: 1.2883 - regu

In [8]:
deepnet.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate))
deepnet.fit(train_dataset, epochs=epochs, verbose=True)

Epoch 1/10
10/10 [==============================] - 0s 5ms/step - RMSE: 2.1617 - loss: 4.1703 - regularization_loss: 0.0000e+00 - total_loss: 4.1703
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - RMSE: 1.1422 - loss: 1.3308 - regularization_loss: 0.0000e+00 - total_loss: 1.3308
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - RMSE: 1.1189 - loss: 1.2616 - regularization_loss: 0.0000e+00 - total_loss: 1.2616
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - RMSE: 1.1180 - loss: 1.2584 - regularization_loss: 0.0000e+00 - total_loss: 1.2584
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - RMSE: 1.1176 - loss: 1.2561 - regularization_loss: 0.0000e+00 - total_loss: 1.2561
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - RMSE: 1.1174 - loss: 1.2546 - regularization_loss: 0.0000e+00 - total_loss: 1.2546
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - RMSE: 1.1172 - loss: 1.2538 - regulariza

In [9]:
age_value=0
gender_value=0

# 예측하고자 하는 데이터를 input_data에 전처리하여 넣기
input_data = tf.constant([[age_value, gender_value]], dtype=tf.float32)

# deepnet 모델을 사용하여 예측 수행
predictions = deepnet.predict(input_data)

# 예측 결과 출력
print(predictions)

1/1 [==============================] - 0s 57ms/step
[[1.1869025]]


In [11]:
# 1. 입력 데이터 준비
# 사용자의 나이와 성별 정보 (예시로 25세 여성)
user_age = 0
user_gender = 0  # 여성은 1, 남성은 0으로 지정

# 2. 입력 데이터 전처리
# 모델이 입력으로 받는 형태로 변환 (2차원 텐서로 변환)
user_features = tf.constant([[user_age, user_gender]], dtype=tf.float32)

# 3. DCN 모델 예측
# crossnet과 deepnet 모델의 예측 값을 합치거나 결합하여 최종 예측을 생성 (여기서는 더해보겠습니다)
crossnet_prediction = crossnet(user_features)
deepnet_prediction = deepnet(user_features)

# 두 모델의 예측 값을 더해서 최종 추천 점수 생성
recommendation_score = crossnet_prediction + deepnet_prediction

# 4. 음악 추천 결과 해석
# 추천 점수가 가장 높은 음악을 추천으로 제공
recommended_music_index = tf.argmax(recommendation_score, axis=1)
recommended_music_title = new_song.iloc[recommended_music_index]['song_name']

print(crossnet_prediction)
print(deepnet_prediction)
print(recommendation_score)
print(recommended_music_index)


print("추천 음악 제목:", recommended_music_title)


tf.Tensor([[0.42173678]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.1869025]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.6086392]], shape=(1, 1), dtype=float32)
tf.Tensor([0], shape=(1,), dtype=int64)
추천 음악 제목: 0    떠나보낼 준비해 둘걸 그랬어
Name: song_name, dtype: object


In [ ]:
# 사용자의 gender와 age 정보 입력
user_gender = 0  # 여성은 1, 남성은 0으로 지정
user_age = 2

# DCN 모델 예측
user_features = tf.constant([[user_age, user_gender]], dtype=tf.float32)
crossnet_prediction = crossnet(user_features)
deepnet_prediction = deepnet(user_features)

# Annoy 모델과 DCN 모델 결과 조합
def combine_recommendation_scores(crossnet_pred, deepnet_pred, annoy_scores):
    # crossnet과 deepnet의 예측 값을 가중치로 사용하여 Annoy 모델의 추천 점수를 업데이트
    combined_scores = crossnet_pred + deepnet_pred + annoy_scores
    return combined_scores

# 곡 추천 함수
def recommend_songs_with_personalization(song_id, annoy_index, tfidf_matrix, crossnet_pred, deepnet_pred, num_neighbors=10):
    # 주어진 곡 ID에 해당하는 가사 벡터를 가져옴
    query_vector = tfidf_matrix[song_id]

    # Annoy 모델을 사용하여 가장 유사한 이웃 곡들을 찾음
    neighbors = annoy_index.get_nns_by_vector(query_vector, num_neighbors)

    # 가장 유사한 이웃 곡들의 인덱스를 반환
    similar_songs = np.array(neighbors)

    # Annoy 모델에서 추천한 곡들의 점수를 계산
    annoy_scores = tf.constant([tfidf_matrix_dense[i] for i in similar_songs], dtype=tf.float32)

    # DCN 모델의 예측 결과를 이용하여 추천 점수를 업데이트
    combined_scores = combine_recommendation_scores(crossnet_pred, deepnet_pred, annoy_scores)

    # 추천 결과를 정렬하여 상위 곡들의 인덱스를 반환
    top_recommended_songs = tf.argsort(combined_scores, direction='DESCENDING')[:num_neighbors]

    return top_recommended_songs

# 예시: id가 0인 곡과 유사한 가사를 가진 곡 추천 (개인화된 추천)
similar_songs_personalized = recommend_songs_with_personalization(26, annoy_index, tfidf_matrix_dense, crossnet_prediction, deepnet_prediction)
print(similar_songs_personalized)


In [ ]:
similar_songs_personalized

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

mat = crossnet._model._dense.kernel
features = ["age", "gender","like"]

plt.figure(figsize=(9,9))
im = plt.matshow(np.abs(mat.numpy()), cmap=plt.cm.Blues)
ax = plt.gca()
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
cax.tick_params(labelsize=10) 
_ = ax.set_xticklabels([''] + features, rotation=45, fontsize=10)
_ = ax.set_yticklabels([''] + features, fontsize=10)

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_recommenders as tfrs

# 데이터 준비
# ... (데이터를 로드하고 전처리하는 코드)

# 하이퍼파라미터 조합 설정
layer_sizes = [[512, 256, 128]]
epochs_list = [10,20,30,40,50]
learning_rates = [0.1,0.2,0.3,0.4,0.5]

# 파라미터 튜닝을 위한 Grid Search
best_model = None
best_accuracy = 0

def compute_accuracy(predictions, labels):
    # logits 형태의 예측값을 클래스로 변환
    predicted_classes = tf.argmax(predictions, axis=1)
    # 실제 레이블과 비교하여 일치하는 샘플의 비율 계산
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_classes, labels), tf.float32))
    return accuracy

for layer_size in layer_sizes:
    for epochs in epochs_list:
        for learning_rate in learning_rates:
            print(f"Training model with layer_size={layer_size}, epochs={epochs}, learning_rate={learning_rate}")

            # 모델 생성
            crossnet = Model(tfrs.layers.dcn.Cross())
            deepnet = Model(
                tf.keras.Sequential([
                    tf.keras.layers.Dense(size, activation="relu") for size in layer_size
                ])
            )

            # 데이터 타입 변경
            train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32), y))

            # 모델 컴파일 및 학습
            crossnet.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate))
            crossnet.fit(train_dataset, epochs=epochs, verbose=False)

            deepnet.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate))
            deepnet.fit(train_dataset, epochs=epochs, verbose=False)

            # 모델 평가
            # ... (평가 데이터에 대해 모델을 평가하는 코드)
            # 예를 들어, test_dataset를 사용하여 평가
            crossnet_accuracy = compute_accuracy(crossnet.predict(train_dataset), train_y)
            deepnet_accuracy = compute_accuracy(deepnet.predict(train_dataset), train_y)
            
            # 평가 결과를 기준으로 최적의 모델 선정
            if crossnet_accuracy > best_accuracy:
                best_accuracy = crossnet_accuracy
                best_model = (layer_size, epochs, learning_rate)

            if deepnet_accuracy > best_accuracy:
                best_accuracy = deepnet_accuracy
                best_model = (layer_size, epochs, learning_rate)

# 최적의 파라미터 출력
print("Best Model:")
print(f"Layer Sizes: {best_model[0]}")
print(f"Epochs: {best_model[1]}")
print(f"Learning Rate: {best_model[2]}")
